In [1]:
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal, assert_almost_equal
from pandas.testing import assert_frame_equal

In [2]:
train = pd.read_csv("resources/train.csv")
test = pd.read_csv("resources/test.csv")
sample_submission = pd.read_csv("resources/sample_submission.csv")

# SLIDE (1) Удаление Nan

Серия задач в данном модуле объединена в одну [большую задачу по предсказанию данных](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview). В ходе выполнения модуля мы будем разбирать определенные техники, которые нужны для ее решения. Настоятельно рекомендуем выполнить все шаги по порядку, тогда в конце вы получите решение большой реальной задачи по МЛ.

Нам даны [данные](https://yadi.sk/d/tcElz6cqSNpPqA) о домах выставленных на продажу. Нам необходимо решить задачу регрессии и предсказать цену продажи дома для $X_{test}$ по данным $X_{train}$ и $y_{train}$. В нашем случае  $y_{train}$ - это столбик `SalePrice`, $X_{train}$ - все остальные столбики.

На вход подается 2 считанных датафрейма **df_train**, **df_test** из файлов без изменений. 

Начальная подготовка:
 * Разделить **df_train** на **X_train**(`pd.Dataframe`) и **y_train**(`pd.Series`).
 * Сконкатенировать **X_train** и **df_test** в **df** по вертикали (можно ориентироваться по столбику `Id` они как раз идут по-порядку). Не забудьте обновить индекс!

Задачи:
 * Заменить в **df** все Nan-ы в категориальных признаках (`object`) на строку `missing`
 * Заменить в **df** все Nan-ы в числовых признаках на 0.

Вернуть из функции измененный **df**.

In [3]:
import pandas as pd

def del_nan(df_train: pd.DataFrame, df_test: pd.DataFrame) -> pd.DataFrame:
    train = df_train.copy()
    y_train = train['SalePrice']
    X_train = train.drop(['SalePrice'], axis='columns')
    df = pd.concat([X_train, df_test], axis=0)
    df = df.reset_index(drop=True)
    df.loc[:, df.dtypes == np.object] = df.loc[:, df.dtypes == np.object].fillna('missing')
    df.loc[:, df.dtypes == np.number] = df.loc[:, df.dtypes == np.number].fillna(0)
    return df

df_without_nan = del_nan(train, test)

# SLIDE (2) Порядковые категории

Вам на вход приходит **df** из предыдущей задачи.

Если внимательно изучить файл `data_description` можно понять, что многие категориальные признаки - порядковые (упорядоченное множество). Значит их можно перевести в осмысленные числа. Значит тут можно воспользоваться `LabelEncoding`.

Ваша задача: заменить в **df** категориальные признаки на числовые, для порядковых признаков.

На выходе возвращаем измененный **df**.

Чтобы слегка упростить вам жизнь, вот вам готовые словари для перевода. Однако к каким столбцам их применять - вы должны выяснить сами, изучив файл `data_description`. Каждый маппинг используется хотя бы 1 раз, а некоторые и не по одному разу.

```python
{'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'missing':0}
{'Gd':4, 'Av': 3, 'Mn': 2, 'No': 1, 'missing': 0}
{'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'missing': 0}
{'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1, 'missing': 0}
{'Fin': 3, 'RFn': 2, 'Unf': 1, 'missing': 0}
{'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'missing': 0}
{'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1, 'missing': 0}
{'Lvl': 4, 'Bnk': 3, 'HLS':2,'Low':1, 'missing': 0}
{'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1, 'missing':0}
{'Gtl':3, 'Mod':2, 'Sev':1, 'missing':0}
{'SBrkr':5, 'FuseA':4, 'FuseF':3, 'FuseP':2, 'Mix':1, 'missing':0}
{'Y':3, 'P':2, 'N':1, 'missing':0}
{'Y':1, 'N':0, 'missing':0} #тут нет ошибки, все так и задумано:)
```

Подсказка: после перевода у вас должно остаться 21 категориальных признака

In [4]:
import pandas as pd

score = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'missing':0}
exposure = {'Gd':4, 'Av': 3, 'Mn': 2, 'No': 1, 'missing': 0}
bsmt_rating = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'missing': 0}
functional = {'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1, 'missing': 0}
finish = {'Fin': 3, 'RFn': 2, 'Unf': 1, 'missing': 0}
fence = {'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'missing': 0}
shape = {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1, 'missing': 0}
flatness = {'Lvl': 4, 'Bnk': 3, 'HLS':2,'Low':1, 'missing': 0}
utilities = {'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1, 'missing':0}
slope = {'Gtl':3, 'Mod':2, 'Sev':1, 'missing':0}
electrical = {'SBrkr':5, 'FuseA':4, 'FuseF':3, 'FuseP':2, 'Mix':1, 'missing':0}
yes_what_no = {'Y':3, 'P':2, 'N':1, 'missing':0}
yes_no = {'Y':1, 'N':0, 'missing':0} #тут нет ошибки, все так и задумано:)

def cat_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['ExterQual'] = df['ExterQual'].map(score)
    df['ExterCond'] = df['ExterCond'].map(score)
    df['BsmtQual'] = df['BsmtQual'].map(score)
    df['BsmtCond'] = df['BsmtCond'].map(score)
    df['HeatingQC'] = df['HeatingQC'].map(score)
    df['KitchenQual'] = df['KitchenQual'].map(score)
    df['FireplaceQu'] = df['FireplaceQu'].map(score)
    df['GarageQual'] = df['GarageQual'].map(score)
    df['GarageCond'] = df['GarageCond'].map(score)
    df['PoolQC'] = df['PoolQC'].map(score)
    
    df['LotShape'] = df['LotShape'].map(shape)
    df['LandContour'] = df['LandContour'].map(flatness)
    df['Utilities'] = df['Utilities'].map(utilities)
    df['LandSlope'] = df['LandSlope'].map(slope)
    df['BsmtExposure'] = df['BsmtExposure'].map(exposure)
    df['BsmtFinType1'] = df['BsmtFinType1'].map(bsmt_rating)
    df['BsmtFinType2'] = df['BsmtFinType2'].map(bsmt_rating)
    df['CentralAir'] = df['CentralAir'].map(yes_no)
    df['Electrical'] = df['Electrical'].map(electrical)
    df['Functional'] = df['Functional'].map(functional)
    df['GarageFinish'] = df['GarageFinish'].map(finish)
    df['PavedDrive'] = df['PavedDrive'].map(yes_what_no)
    df['Fence'] = df['Fence'].map(fence)
    return df

df_label_encoded = cat_to_num(df_without_nan)

# SLIDE (1) One hot encoding

Теперь разберемся с непорядковыми категориальными признаками.

Для начала заметим признак `MSSubClass`, у которого тип `int64`, но если посмотреть в описание `data_description` можно понять, что это - категориальный признак. 
 * Измените тип признака `MSSubClass` с `int64` на `object`

Теперь можно сделать `One hot encoding`:
 * Найдите все колонки с категориальными признаками и составьте из них отдельный **df_oh** `pd.DataFrame` (индекс сохранить прежний)
 * Применить к полученному фрейму **df_oh** функцию [`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) (Реализует `One Hot Encoding`)
 * Удалить категориальные колонки из **df** и добавить справа к **df** фрейм с `One Hot Encoding`
 
Вернуть из функции **df**

Подсказка: итого должно получится 241 колонка из них 183 пришли из `One hot encoding` фрейма

In [5]:
import pandas as pd
import numpy as np

def one_hot(df: pd.DataFrame) -> pd.DataFrame:
    df['MSSubClass'] = df['MSSubClass'].astype('object')
    df_oh = df.select_dtypes('object')
    df_oh = pd.get_dummies(df_oh)
    df = df.loc[:, df.dtypes != np.object]
    df = pd.concat([df, df_oh], axis='columns')
    return df

df_one_hot = one_hot(df_label_encoded)

# SLIDE (2) Некоррелирующие признаки 

Мы разобрались с категориальными признаками, теперь разберемся с числовыми.
Для числовых признаков можно посчитать корреляцию с правильным ответом. Если признаки слабо коррелируют, то они нам не нужны. Например колонка `Id` явно никак не влияет на стоимость дома.

Вам на вход передается изначальный **df_train** и **df** полученный из предыдущей задачи.

Ваша задача: 
 * найти корреляцию всех **числовых** признаков **df_train** с признаком `SalePrice` с помощью `pd.corr`
 * если абсолютное значение корреляции признака с `SalePrice` меньше $0.05$ - удалите этот признак из **df**

Верните измененный **df** и столбец корреляции признаков с признаком `SalePrice` упорядоченный по убыванию. Начало столбца корреляции выглядит следующим образом:

|               |SalePrice |
|---------------|----------|
|**SalePrice**  |1.000000  |
|**OverallQual**|0.790982  |
|**GrLivArea**  |0.708624  |
|**GarageCars** |0.640409  |
|**GarageArea** |0.623431  |
|**TotalBsmtSF**|0.613581  |
|**1stFlrSF**   |0.605852  |
|**FullBath**   |0.560664  |

Всего должно получиться 37 числовых признаков.

Подсказка: как мы помним столбец`MSSubClass` - категориальный, уберите его из **df_train** при рассмотрении корреляции.

Подсказка: всего придется удалить 8 признаков (включая `Id`). В **df** останется 233 признака.

In [6]:
import numpy as np

def correlation(df: pd.DataFrame, df_train: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    train = df_train.copy()
    del train['MSSubClass']
    train_number = train.select_dtypes(include='number')
    corr_matrix = train_number.corr()
    corr_sale_price = corr_matrix.loc[:, 'SalePrice'].sort_values(ascending=False)
    bad_corr_columns = corr_sale_price[abs(corr_sale_price) < 0.05].index
    df = df.drop(bad_corr_columns, axis='columns')
    return df, pd.DataFrame(corr_sale_price, columns=['SalePrice'])
    

df_with_corr, corr = correlation(df_one_hot, train)

# SLIDE (1) Feature Engineering

Давайте нагенерируем несколько фич во входном фрейме **df**:
 * `TotalArea` = `TotalBsmtSF` + `1stFlrSF` + `2ndFlrSF` + `GrLivArea` + `GarageArea`
 * `YearAverage` = (`YearRemodAdd` + `YearBuilt`) / 2
 * `LiveAreaQual` = `OverallQual` * `GrLivArea`

На выход отправьте **df** c тремя новымим столбиками. столбцы должны идти в том же порядки что указаны в списке в хвосте **df**.

In [7]:
import pandas as pd

def feature_en(df: pd.DataFrame) -> pd.DataFrame:
    df['TotalArea'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF'] + df['GrLivArea'] + df['GarageArea']
    df['YearAverage'] = (df['YearRemodAdd'] + df['YearBuilt']) / 2
    df['LiveAreaQual'] = df['OverallQual'] * df['GrLivArea']
    return df

df_featured = feature_en(df_with_corr)

# SLIDE (1) Scaling с выбросами

У стандартного и нормального масштабирования есть одна проблема: она учитывает все признаки, даже те, которые изначально некорректны (шум, выбросы). Чтобы избавитьться от шумов и выбросов и корректно масштабировать выборку необходимо использовать [RobustScaling]((https://scikit-learn.org/0.18/auto_examples/preprocessing/plot_robust_scaling.html).

Ваша задача - отмасштабировать полученный фрейм с помощью `RobustScaler`. И вернуть отмасштабированный массив (да, скалирование возвращает массив, а не DataFrame).

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler

def scaling(df: pd.DataFrame) -> np.array:
    robust_scaler = RobustScaler()
    return robust_scaler.fit_transform(df)

X_scaled = scaling(df_featured)
print(X_scaled)

[[ 0.05714286 -0.24511241  0.         ...  0.27028555  0.63414634
   0.54896142]
 [ 0.48571429  0.03592375  0.         ... -0.05654509 -0.02439024
  -0.1760633 ]
 [ 0.14285714  0.43914956 -1.         ...  0.42635001  0.59756098
   0.63666337]
 ...
 [ 2.77142857  2.57746823  0.         ... -0.05541419  0.02439024
  -0.41543027]
 [-0.02857143  0.24144673  0.         ... -0.84478372  0.36585366
  -0.62479393]
 [ 0.31428571  0.04252199  0.         ...  0.73508623  0.40243902
   0.88361358]]


# SLIDE (2) Смешанная модель

Отлично, теперь мы готовы обучать модель! Осталось изучить последний интересный трюк - смешанные модели.

Возьмем [2 регрессии](https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b):
 * Ridge
 * Lasso
 
Найдем оптимальные значения $\alpha$ для обеих регрессий с помощью GridSearch.

Теперь отправим обе модели с наилучшими параметрами в класс указанный снизу. Это класс смешения моделей. В нем параметр $\beta \in [0,1]$ - это коэффициент, с которым берется ответ одного классификаторв, а ответ второго - с коэффициентом $(1 - \beta)$. Такая техника нередко позволяет добиться лучших результатов, чем одна модель.

Теперь найдем наилучшее $\beta$ для смешенной модели также с помощью GridSearch. Осталось получить **y_pred** с помощью наилучшей смешанной модели.

На вход вы получаете **X_scaled** из предыдущей задачи и **df_train** начальный. Мы подготовили за вас **X_train**, **y_train** и **X_test**. 

В задаче необходимо минимизировать метрику `neg_mean_squared_log_error`. Для удобства мы возьмем `np.log1p(y_train)` и будем минимизировать метрику `neg_mean_squared_error`. Эту метрику необходимо минимизировать у всех 3-х GridSearch.

На выход отправьте GridSearch объект смешанной модели, а также результат **y_test**. (Не забудьте его проэкспоненциировать).

Мы выдаем вам ориентировачные параметры для каждого GridSearch. Вы можете увеличить перебор, чтобы получить лучшую модель.
```python
params_ridge = {'alpha': np.arange(1, 20)}
params_lasso = {'alpha': np.logspace(-4, 3, num=8, base=10)}
params_blend = {'beta': np.linspace(0, 1, 11)}
```

Первые 2 GridSearch **не нужно** писать в функции: они могут работать достаточно долго и превысят лимит работы задачи на сервере. Найдите у себя локально наилучшие параметры и уже с этими параметрами создайте смешанную модель внутри функции. 

Также не нужно сильно увеличивать перебор для $\beta$ - того, что есть, более чем достаточно.

P.S. Осталось сохранить файл с **y_test** и отправить его в [соревнование](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submit). Улучшайте свои результы пробуя другие модели и другие параметры.


In [18]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso

class BlendRegressor(BaseEstimator): # предок класса классификаторов, чтобы можно было засунуть в GridSearch
    def __init__(self, clf1, clf2, beta=0.5):
        self.clf1 = clf1 
        self.clf2 = clf2
        self.beta = beta #параметр смешивания

    def fit(self, X, y): #обучаем классификатор
        self.X_ = X 
        self.y_ = y 
        self.clf1.fit(X, y) 
        self.clf2.fit(X, y)
        return self

    def predict(self, X): #возвращаем значения 
        return self.clf1.predict(X) * self.beta + self.clf2.predict(X) * (1 - self.beta)
    

def get_ridge_gs():
    params_ridge = {'alpha': np.arange(1, 20)}
    
    return GridSearchCV(
        estimator = Ridge(),
        param_grid = params_ridge,
        cv = 5,
        scoring = 'neg_mean_squared_error'
    )

def get_lasso_gs():
    params_lasso = {'alpha': np.logspace(-4, 3, num=8, base=10)}
    
    return GridSearchCV(
        estimator = Lasso(),
        param_grid = params_lasso,
        cv = 5,
        scoring = 'neg_mean_squared_error'
    )

    
def learning(X_scaled: np.array, df_train: pd.DataFrame) -> (GridSearchCV, np.array):
    X_train = X_scaled[0: len(df_train),]
    X_test  = X_scaled[len(df_train): len(X_scaled)]
    y_train = np.log1p(df_train['SalePrice'])

    params_blend = {'beta': np.linspace(0, 1, 11)}
    
    blend_gs = GridSearchCV(
        estimator = BlendRegressor(Ridge(alpha=5), Lasso(alpha=0.001)),
        param_grid = params_blend,
        cv = 5,
        scoring = 'neg_mean_squared_error'
    )
    
    blend_gs.fit(X_train, y_train)
    y_test = blend_gs.predict(X_test)
    
    return blend_gs, np.exp(y_test) - 1

# Calculate best params for Ridge and Lasso models
'''
X_train = X_scaled[0: len(train),]
X_test  = X_scaled[len(train): len(X_scaled)]
y_train = np.log1p(train['SalePrice'])

ridge_gs = get_ridge_gs()
ridge_gs.fit(X_train, y_train)
print(ridge_gs.best_params_)

lasso_gs = get_lasso_gs()
lasso_gs.fit(X_train, y_train)
print(lasso_gs.best_params_)
'''

learning(X_scaled, train)

(GridSearchCV(cv=5, error_score='raise-deprecating',
              estimator=BlendRegressor(beta=0.5,
                                       clf1=Ridge(alpha=5, copy_X=True,
                                                  fit_intercept=True,
                                                  max_iter=None, normalize=False,
                                                  random_state=None,
                                                  solver='auto', tol=0.001),
                                       clf2=Lasso(alpha=0.001, copy_X=True,
                                                  fit_intercept=True,
                                                  max_iter=1000, normalize=False,
                                                  positive=False,
                                                  precompute=False,
                                                  random_state=None,
                                                  selection='cyclic', tol=0.0001,
               